In [12]:
import torch
import torch.nn as nn
from MobileNetV2 import mobilenet_v2
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np
from MobileNetV3 import mobilenetv3

In [13]:
save_path = 'modelv3.pth'
model = mobilenetv3(mode="large")
# model = mobilenet_v2()
model.load_state_dict(torch.load(save_path))
model = model.cuda(2)

In [14]:
# 超参数
input_size = 224
batch_size = 27
n_worker = 8
gpu_id = 2

In [15]:
# 输入数据transform
data_transform = transforms.Compose([
        # transforms.RandomResizedCrop(input_size, scale=(0.2, 1.0)),
        # transforms.Resize([input_size, input_size]),
        transforms.ToTensor(),
])
test_path = "image/test_image"
test_dataset = datasets.ImageFolder(test_path, transform=data_transform)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, 
    num_workers=n_worker, pin_memory=True)

In [18]:
ans = 0
idx = 0
model.eval()
with torch.no_grad():
    for data, label in test_loader:
        data, label = data.cuda(2), label.cuda(2)
        output = model(data)
        preds = torch.argmax(output, 1)
        unique_values, counts = torch.unique(preds, return_counts=True)
        resp = unique_values[counts.argmax()]

        if resp.item() == idx:
            # print(f'person {idx} identify success!')
            ans += 1
            # print(f'person {idx} identify fail')
        idx += 1
ans / 20

0.45

In [17]:
ans

9